In [2]:
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

# Retrieve the data
mnist = fetch_openml("mnist_784")
X, y = mnist["data"], mnist["target"]
# print('X', X)
# print('y', y)

# Separate the training and test sets
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# Build the classifier and fit
sgd_clf = SGDClassifier(random_state=42)
s

# Perform cross validation
print("before")
score = cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")
print("after")
print("score", score)

before
after
score [0.87365 0.85835 0.8689 ]
